In [ ]:
# Installations
!pip install transformers
!pip install datasets
!pip install openpyxl

In [ ]:
# Imports
from datasets import load_dataset
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
import openpyxl
from transformers import AutoTokenizer
import csv
from csv import reader

In [ ]:
# Loading the dataset
dataset_file = 'dataset.csv'
dataset = load_dataset('csv', data_files=dataset_file, split='train')
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']

In [ ]:
# Tokenization
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize(batch):
    tokenized_input = tokenizer(batch['source'], padding='max_length', truncation=True, max_length=900)
    tokenized_label = tokenizer(batch['target'], padding='max_length', truncation=True, max_length=900)
    tokenized_input['labels'] = tokenized_label['input_ids']
    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=512)
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

In [ ]:
# Training
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
output_dir = 'output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=6,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=1, 
    learning_rate=0.0001,
    evaluation_strategy='steps', 
    remove_unused_columns=True, 
    run_name='run_name', 
    logging_steps=1000, 
    eval_steps=1000,
    adam_beta1=0.7,
    adam_beta2=0.9,
    adam_epsilon=1.8e-8 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
trainer.save_model(output_dir + '/model')

In [ ]:
# Evaluation
model_dir = 'output/model'
output_dir = 'output'
model = BartForConditionalGeneration.from_pretrained(model_dir)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred[0], axis=2)
    accuracy = accuracy_score(y_true=labels[0], y_pred=pred[0])
    recall = recall_score(y_true=labels[0], y_pred=pred[0], average='macro')
    precision = precision_score(y_true=labels[0], y_pred=pred[0], average='macro')
    f1 = f1_score(y_true=labels[0], y_pred=pred[0], average='macro')
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

pred_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=1,
    remove_unused_columns=True,
    eval_accumulation_steps=1
)

trainer = Trainer(model=model, args=pred_args, compute_metrics=compute_metrics)

preds, labels, metrics = trainer.predict(val_dataset)
preds_tokens = preds[0].argmax(axis=2)
print(metrics)

decoded_sources = []
for row in val_dataset:
    decoded_sources.append(tokenizer.decode(row['input_ids']))

decoded_preds = [tokenizer.decode(pred) for pred in preds_tokens]
decoded_labels = [tokenizer.decode(label) for label in labels]

output = pd.DataFrame({'Source Text': decoded_sources, 'Target Text': decoded_labels, 'Generated Text': decoded_preds})
output.to_csv(output_dir + "/predictions.csv")

In [ ]:
dataset_file = 'dev_dataset.csv'
dev_dataset = load_dataset('csv', data_files=dataset_file, split='train')

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize(batch):
    tokenized_input = tokenizer(batch['source'], padding='max_length', truncation=True, max_length=900)
    tokenized_label = tokenizer(batch['target'], padding='max_length', truncation=True, max_length=900)
    tokenized_input['labels'] = tokenized_label['input_ids']
    return tokenized_input

dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=len(dev_dataset))

In [ ]:
#Predict with beam search
model_dir = 'output/model'
output_dir = 'output'
model = BartForConditionalGeneration.from_pretrained(model_dir)

pred_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=1,
    remove_unused_columns=True,
    eval_accumulation_steps=1,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(model=model, args=pred_args, tokenizer=AutoTokenizer.from_pretrained('facebook/bart-base'))

preds, labels, metrics = trainer.predict(dev_dataset, num_beams=3, max_length=900)
preds_tokens = preds

decoded_sources = []
for row in dev_dataset:
    decoded_sources.append(tokenizer.decode(row['input_ids']))

decoded_preds = [tokenizer.decode(pred) for pred in preds_tokens]
decoded_labels = [tokenizer.decode(label) for label in labels]

output = pd.DataFrame({'Source Text': decoded_sources, 'Target Text': decoded_labels, 'Generated Text': decoded_preds})
output.to_csv(output_dir + "/beam_predictions.csv")